## Problem 1-4 : NorthWinddatset https://relational.fit.cvut.cz/dataset/Northwind

How to download the dataset

To download the dataset you need to create connection in your mysql workbench

The datasets are publicly available directly from MariaDB database.

* Open your favourite MariaDB client (MySQL Workbench)
Use following credentials:
```
    hostname: relational.fit.cvut.cz
    port: 3306
    username: guest
    password: relational
```
Drive link: https://drive.google.com/drive/folders/1FMi6g1KDHbn17ikJSi7bE9-THywlxhdu?usp=sharing

###`Q-1`: Rank Employee in terms of revenue generation. Show employee id, first name, revenue, and rank


###`Q-2`: Show All products cumulative sum of units sold each month.


###`Q-3`: Show Percentage of total revenue by each suppliers



###`Q-4:` Show Percentage of total orders by each suppliers



### `Q-5`:Show All Products Year Wise report of totalQuantity sold, percentage change from last year.





In [ ]:
# q1
# -- Rank Employee in terms of revenue generation. Show employee id, first name, revenue, and rank

select  t3.EmployeeID,t3.FirstName,sum(t1.UnitPrice),
rank() over( order by sum(t1.UnitPrice * t1.Quantity) desc) as 'Rank'
 from nw_order_details t1
join
nw_orders t2
on t1.OrderID = t2.OrderID
join
nw_employees t3
on t2.EmployeeID = t3.EmployeeID

group by t3.EmployeeID,t3.FirstName


# q2
# -- Show All products cumulative sum of units sold each month.


select *,sum(monthly_quantity) over(partition by ProductName order by order_date rows between unbounded preceding and current row ) as 'cum_quantity' from (
select t2.ProductName,monthname(t3.OrderDate) as 'order_date',sum(Quantity) as 'monthly_quantity' from nw_order_details t1
join
nw_products t2
on t1.ProductID = t2.ProductID
join
nw_orders t3
on t1.OrderID = t3.OrderID
group by t2.ProductName,monthname(t3.OrderDate)
order by t2.ProductName,monthname(t3.OrderDate))t

order by ProductName asc ,month(order_date)


# q3
# -- Show Percentage of total revenue by each suppliers
select *,(total_revenue/sum(total_revenue) over())*100 as '%revenue_by_supplier' from (
select t4.SupplierID,t4.ContactName,sum(t1.UnitPrice * t1.Quantity) as 'total_revenue' from nw_order_details t1
join
nw_products t2
on t1.ProductID = t2.ProductID
join
nw_orders t3
on t1.OrderID = t3.OrderID
join
nw_suppliers t4
on t2.SupplierID = t4.SupplierID
group by t4.SupplierID,t4.ContactName) t


#q4
# -- Show Percentage of total orders by each suppliers

select *,total_orders_per_person/sum(total_orders_per_person) over() * 100 as '%orders_by_supplier' from (
select t4.SupplierID,t4.ContactName,count(distinct t3.OrderID) as 'total_orders_per_person' from nw_order_details t1
join
nw_products t2
on t1.ProductID = t2.ProductID
join
nw_orders t3
on t1.OrderID = t3.OrderID
join
nw_suppliers t4
on t2.SupplierID = t4.SupplierID
group by t4.SupplierID,t4.ContactName) t


#q5
# -- Show All Products Year Wise report of totalQuantity sold, percentage change from last year.

select *,lag(year_by_year) over(partition by ProductName),
((year_by_year - lag(year_by_year) over(partition by ProductName))/lag(year_by_year) over(partition by ProductName))*100 as '%yearly_increase'
 from (
select t2.ProductName,year(t3.OrderDate) as 'yearly' ,
sum(sum(Quantity)) over(partition by t2.ProductName,year(t3.OrderDate)) as 'year_by_year'
from nw_order_details t1
join
nw_products t2
on t1.ProductID = t2.ProductID
join
nw_orders t3
on t1.OrderID = t3.OrderID
group by t2.ProductName,year(t3.OrderDate)) t




## Problem 6 - 15

For the next problems, you can get the dataset as well as the details [here](https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug_clean.csv) (Use the *cleaned* csv file).

### **`Problem-6:`** For each condition, what is the average satisfaction level of drugs that are "On Label" vs "Off Label"?


### **`Problem-7:`** For each drug type (RX, OTC, RX/OTC), what is the average ease of use and satisfaction level of drugs with a price above the median for their type?


### **`Problem 8:`** What is the cumulative distribution of EaseOfUse ratings for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and cumulative distribution. (Use the built-in method and the manual method by calculating on your own. For the manual method, use the "ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" and see if you get the same results as the built-in method.)


### **`Problem 9:`** What is the median satisfaction level for each medical condition? Show the results in descending order by median satisfaction level. (Don't repeat the same rows of your result.)



### **`Problem 10:`** What is the running average of the price of drugs for each medical condition? Show the results in ascending order by medical condition and drug name.



### **`Problem 11:`** What is the percentage change in the number of reviews for each drug between the previous row and the current row? Show the results in descending order by percentage change.



### **`Problem 12:`** What is the percentage of total satisfaction level for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and percentage of total satisfaction.



### **`Problem 13:`** What is the cumulative sum of effective ratings for each medical condition and drug form combination? Show the results in ascending order by medical condition, drug form and the name of the drug.



### **`Problem-14:`** What is the rank of the average ease of use for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by rank and drug type.



### **`Problem-15:`** For each condition, what is the average effectiveness of the top 3 most reviewed drugs?

In [ ]:
# problem 6
# -- For each condition, what is the average satisfaction level of drugs that are "On Label" vs "Off Label

select *,avg(Satisfaction) over(partition by `Condition`,Indication) from drug



# problem 7

# -- For each drug type (RX, OTC, RX/OTC), what is the average ease of use and satisfaction 
# -- level of drugs with a price above the median for their type?
select * from (
select *,avg(Satisfaction) over(partition by Type) as 'avg_sat' ,
avg(EaseOfUse) over(partition by Type) as 'avg_ease' ,
percentile_disc(0.5) within group(order by price) over(partition by Type) as 'median'
from drug) t
where price > median



# problem 8
# --  What is the cumulative distribution of EaseOfUse ratings for each drug type (RX, OTC, RX/OTC)? 
# -- Show the results in descending order by drug type and cumulative distribution. 
# -- (Use the built-in method and the manual method by calculating on your own. 
# -- For the manual method, use the "ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" 
#  -- and see if you get the same results as the built-in method
 
 select *,cume_dist()  over(partition by `Type` order by EaseOfUse ) from drug



# problem 9
# -- What is the median satisfaction level for each medical condition? 
# -- Show the results in descending order by median satisfaction level. 
# -- (Don't repeat the same rows of your result
select `Condition`,median_sat_per_condition from (
select *,percentile_disc(0.5) within group(order by Satisfaction) over(partition by `Condition`) as 'median_sat_per_condition' from drug ) t
group by `Condition`,median_sat_per_condition
order by median_sat_per_condition desc


# problem 10
# -- What is the running average of the price of drugs for each medical condition? 
# -- Show the results in ascending order by medical condition and drug name
select * from (
select *,avg(Price) over(partition by `Condition` order by price rows between unbounded preceding and current row)  as 'running_avg' from drug ) t
order by `Condition`,Drug



#problem 11
# -- What is the percentage change in the number of reviews for each drug between the previous row 
# -- and the current row? Show the results in descending order by percentage change.
select * from (
select *,lag(Reviews) over(partition by Drug order by Reviews rows between 1 preceding and current row) as 'check',
((Reviews-lag(Reviews) over(partition by Drug order by Reviews rows between 1 preceding and current row))/lag(Reviews) over(partition by Drug order by Reviews rows between 1 preceding and current row))*100 as 'percent_change'

from drug) t
order by percent_change desc




# problem 12
# -- What is the percentage of total satisfaction level for each drug type (RX, OTC, RX/OTC)? 
# -- Show the results in descending order by drug type and percentage of total satisfaction.
select *,sum_sat/sum(sum_sat) over() * 100 as 'total_satisfaction' from (
select `Type`,sum(Satisfaction) as 'sum_sat' from drug
where `Type` in ('RX', 'OTC', 'RX/OTC')
group by `Type`) t
order by `Type` desc , total_satisfaction desc



# problem 13
# -- What is the cumulative sum of effective ratings for each medical condition and drug form combination? 
# -- Show the results in ascending order by medical condition, drug form and the name of the drug.
select * from (
select *,sum(Effective) over(partition by `Condition`,Form order by Drug rows between unbounded preceding and current row) as 'cum_sum' from drug) t
order by t.`Condition`,t.Form,t.Drug



# problem 14
#  -- What is the rank of the average ease of use for each drug type (RX, OTC, RX/OTC)?
#  -- Show the results in descending order by rank and drug type.
 select *,dense_rank() over(order by avg_type desc) from (
 select `Type`,avg(EaseOfUse) as 'avg_type' from drug
 where `Type` in ('RX', 'OTC', 'RX/OTC')
 group by `Type`) t
 
 
 # problem 15
#  -- For each condition, what is the average effectiveness of the top 3 most reviewed drugs?
select `Condition`,avg(avg_sat) from (
select *,dense_rank() over(partition by `Condition` order by total_review desc) as 'rank' from (
select `Condition`,Drug,avg(Satisfaction) as 'avg_sat' ,sum(Reviews) as 'total_review' from drug
group by `Condition`,Drug ) t ) a 
where a.rank < 4
group by `Condition`
